In [1]:
import numpy as np
import pandas as pd
from textblob import TextBlob

In [2]:
from datetime import date
from sklearn.feature_extraction.text import CountVectorizer


# Outline of pre-processing steps:
1. Remove user accounts - done
2. Make every tweet lowercase - done
3. Remove special characters, including hashtags - done
4. Spellcheck - bail 12 hours of CPU time
5. Make one df of tweets that only contain desired keywords - bail not run
6. Aggregate time by hour

In [3]:
# Read in data:
accounts = pd.read_excel("AccountByFreq.xlsx")
thirty_plus = accounts[accounts['Unnamed: 1']>30]

In [4]:
tweets = pd.read_csv("YInt.csv")
tweets.dropna(inplace=True)
#tweets.set_index('account')
bad_list = ['J0rdanWantsBacon','handle','JordanWantsBac0n','______3333_____','CantonCoordon2','Opportunities1','Opportunities2','Syndicated4','Syndicated5','Syndicated348']

In [8]:
print(tweets.shape)
for b in bad_list:
    tweets = tweets[tweets['account']!=b]
tweets['message'] = tweets['message'].str.lower()
tweets['message'] = tweets['message'].str.replace(r"[\"\',#$.!]", '')
tweets = tweets[tweets['location'] != "UNKNOWN"]
tweets = tweets[tweets['location'] != "<Location with-held due to contract>"]
tweets['time'] = pd.to_datetime(tweets['time'])
tweets['time'] = tweets['time'].map(round_date_to_unix)
tweets['time'] = tweets
tweets.drop(columns="account",inplace=True)
print(tweets.shape)

(40970, 4)
(40970, 3)


In [12]:
retweets = tweets.copy()

In [7]:
def spell_check(x):
    a = TextBlob(x).correct()
    return str(a)

In [6]:
def round_date_to_unix(x):
    c = x.round("H")
    c = int(c.timestamp())*1000
    return c

In [9]:
#tweets['message'] = tweets['message'].map(spell_check,na_action='ignore')

In [10]:
# test = tweets.head(20)
# test['message'].map(spell_check,na_action='ignore')

In [11]:
#name_list = thirty_plus['Account'].values.tolist()
# for name in name_list:
#     account_tweets = tweets[tweets['account']==name]['message']
#     vectorizer = CountVectorizer(strip_accents = "unicode",lowercase = True, stop_words = 'english',max_features = 10,ngram_range = (1,1))
#         # tokenize and build vocab
#     res = vectorizer.fit_transform(account_tweets)
#         # summarize encoded vector
#     a = np.sum(res.toarray(),axis=0)
    
#     res_list = []
#     for key, value in vectorizer.vocabulary_.items(): 
#             final_dict = {}
#             final_dict["name"] = name 
#             final_dict["word"] = key
#             final_dict["value"] = a[value]
#             res_list.append(final_dict)
#             #print(final_dict,"\n")


In [438]:
tweets['time'].value_counts()

1586358000000    2182
1586354400000    1641
1586185200000    1182
1586336400000     982
1586372400000     832
                 ... 
1586311200000      12
1586131200000      10
1586134800000       8
1586221200000       6
1586138400000       6
Name: time, Length: 109, dtype: int64

In [439]:
a = tweets.groupby(["time"])

In [443]:
curr = {}
for c,d in a.__iter__():
    curr[c] = a.get_group(c).to_dict(orient='records')

In [446]:
import json

dict = curr

json = json.dumps(dict)
f = open("data_by_time.json","w")
f.write(json)
f.close()

In [18]:
bool_list = retweets['message'].str.startswith("re:")

In [21]:
curr = retweets[bool_list]

In [23]:
rt_time = curr.groupby(["time"])

In [45]:
rts = {}
i = 0
for c,d in rt_time.__iter__():
    values = d['message'].value_counts().keys().tolist()
    counts = d['message'].value_counts().tolist()
    curr_list = []
    for v, n in zip(values, counts):
        curr_dict = {}
        curr_dict["message"] = v
        curr_dict["value"] = n
        curr_list.append(curr_dict)
    rts[c] = curr_list[:5]

In [47]:
print(rts)

{1586149200000: [{'message': 're: let me know when in it just reminded him to be doing right now hikz so i still think you have shown more', 'value': 7}, {'message': 're: 860mb of mp3 promos to listen hahah i know right anything sooo amazzzingi cant wait to go to it', 'value': 5}, {'message': 're: i think its being judged by various people and a she rossi 1 aswell eee what a good weekend now looking forward to have open while i fold laundry no way thats how generous people', 'value': 4}, {'message': 're: i am at home workable =d now for a while she glad that she template extensions-&gt;template manager-&gt;select template-&gt;edit-&gt;edit css set colours', 'value': 1}], 1586152800000: [{'message': 're: it i was so worthless lol fork that something loved that movie', 'value': 25}, {'message': 're: i am at home workable =d now for a while she glad that she template extensions-&gt;template manager-&gt;select template-&gt;edit-&gt;edit css set colours', 'value': 22}, {'message': 're: i th

In [43]:
z = [1,2,3,4,5,6]

In [44]:
z[:5]

[1, 2, 3, 4, 5]